# `multiprocessing`

Parallel computation on a single machine in Python
- one of my most important tools
- Python standard library


## Python standard library parallel computation ecosystem

[Multiprocessing Vs. Threading In Python - Sid Panjwani](https://timber.io/blog/multiprocessing-vs-multithreading-in-python-what-you-need-to-know/)

`threading` - uses threads (same memory space)
- helps with network issues

`multiprocessing` - uses processes (different memory space)
- help with compute issues

How does this relate to CPU cores
- CPU cores are fixed (usually 4-16 in laptops - depends on your physical hardware)
- more cores = true parallelism (opposed to the very fast task switching done by the OS
- your computer can have many threads and many processes (depends on the OS)
- the OS will schedule these threads/processes to available cores
- a single thread consumes an entire core

[Multithreading and multicore differences](https://stackoverflow.com/questions/11835046/multithreading-and-multicore-differences)

*But my CPU core has two threads*
- this is a different use of the term (the hardware thread)
- CPU having threads allows a core to run thread in parallel, as if there were multiple cores
- known as hyperthreading


## Why do we need `multiprocessing`?

Python has a Global Interpreter Lock (GIL) that prevents parallelizing computation across multiple cores
- Python is not thread safe
- requires a lock when accessing an object (a form of memory management)


## What can be hard in multiprocessing?

Sharing things between processes
- solution = don't use it in this way
- make every process independent
- a functional style = no interaction (because interaction = side effects!)


## `multiprocessing` 101

We map functions to data
- but in parallel!

First let's do a simple `map` in Python:

In [ ]:
import time
import numpy as np

def subtract(x, sleep=0.01):
    time.sleep(sleep)
    return x*x

data = np.random.uniform(0, 100, size=100).tolist()
st = time.time()
result = list(map(subtract, data))
print(time.time() - st)

Let's parallelize this using `multiprocessing`:

In [ ]:
from multiprocessing import Pool

num_process = 8
st = time.time()
with Pool(num_process) as pool:
    out = pool.map(subtract, data)
    
print(time.time() - st)

A common use case is to have arguments for the function being mapped:

In [ ]:
from functools import partial

st = time.time()
with Pool(num_process) as p:
    rewards = p.map(partial(subtract, sleep=0.0), data)
print(time.time() - st)

Note that when we remove our sleep, the non-mulitprocessing `map` is faster:

In [ ]:
st = time.time()
result = list(map(partial(subtract, sleep=0.0), data))
print(time.time() - st)

Distributed computation has overhead (fixed + variable) 
- make sure your function runs long enough to justify it

## Evolutionary methods with `multiprocessing`

*At least let me have a little fun!*

Context on evolutionary algorithms from the Four Competences (on whiteboard)
- [more context here](https://towardsdatascience.com/daniel-c-dennetts-four-competences-779648bdbabc?source=friends_link&sk=15fe38a0971a25c0ddb028aec05109a4)
- **neuroevolution** = using evolutionary methods to find the parameters (or even architecture) of a neural network

Computational evolutionary methods
- general, black box optimization
- gradient free 
- work in challenging cost functions (non-linear, discontinuous etc)
- can be parallelized

Sample inefficient
- because they learn from a weak learning signal (fitness / total episode reward)
- don't learn from state / reward transitions that occur during an episode

## Generate, test & select

Evolutionary improvement occurs through a **generate, test & select loop**
- substrate independent

Our algorithm will:
- generate a population of parameters (neural network weights and biases)
- test these parameters in the `mountaincar` environment
- select the best performing set of parameters to use in the next generate step

Let's first setup the code for the forward pass of a neural net.  We aren't going to do any backprop, so we can do it all in `numpy`:

In [ ]:
!pip install gym -q

import gym
from evolution import initialize_parameters, episode, forward

#  need this to get the size of the nn input & outpun
env = gym.make('MountainCarContinuous-v0')

i_size = env.observation_space.shape[0]
h_size = 10
o_size = env.action_space.shape[0]
params = initialize_parameters(i_size, h_size, o_size)

We end up with a dictionary of parameters with random weights:

In [ ]:
params.keys()

We can use the function forward to select an action using these parameters & a randomly sampled observation:

In [ ]:
action = forward(env.observation_space.sample(), params)
action

Below the machinery for saving and loading parameters is given - this is so you can run `python render_mountaincar.py` when you agent is learning:

In [ ]:
from evolution import save_params, load_params

params = initialize_parameters(i_size, h_size, o_size)
save_params(params, 1)
params = load_params(1)

Now you should be able to run (in a shell - will break your notebook kernel):

```bash
$ python render.py --agent_id 1
```

![](assets/car.png)

## The components of a simple evolutionary algorithm

Above we have outlined the code we need to do the test step of generate/test/select.  Now we will outline the code for the generate & select steps.

For the first generation, the loop is to **generate** a population with random weights:

In [ ]:
pop_size = 32
pop = [initialize_parameters(i_size, h_size, o_size) for _ in range(pop_size)]

**Test** the population in the environment:

In [ ]:
results = list(map(episode, pop))
print(np.mean(results))
assert len(results) == pop_size

**Select** the best performing parameters:

In [ ]:
best = pop[np.argmax(results)]

We are now on the second generation.  

We still sample a new generation randomly, but now we use the results of the first generation to create the distribution to sample from.

Below we create a new generation, using the best performing member to estimate the mean + an identity covariance matrix:

In [ ]:
def sample_params(best):
    p = {}
    for k, v in best.items():
        flat = v.flatten()
        new = np.random.multivariate_normal(flat, np.eye(flat.shape[0]), size=1)
        p[k] = new.reshape(v.shape)
    return p

pop = [sample_params(best) for _ in range(pop_size)]

In [ ]:
results = list(map(episode, pop))
print(np.mean(results))

## Practical

Take the components above and put them together:
- implement an evolutionary method using `map` (single core)
- implement an evolutionary method using `pool.map` (multi-core)